# Chimaps in a few lines of code

Based on the posts [Chimaps in a few lines of code](https://topotoolbox.wordpress.com/2017/08/18/chimaps-in-a-few-lines-of-code-final/) by Wolfgang Schwanghart (2017).

In [ ]:
import topotoolbox as topo
import numpy as np
import matplotlib.pyplot as plt

## Introduction

Since Willett’s et al. (2014) paper on the growth and decay of drainage basins, chimaps have become a popular tool to illustrate the direction of divide migration. Whether a drainage basin is a victim or aggressor, i.e., will loose or gain catchment area in the future, is indicated by the variable chi in the vicinity of the divides. chi (Χ) is a transformed horizontal coordinate measured from the outlet of a drainage basin. The transformation uses upslope area to linearize the concave upward profile for a well-chosen value of the mn-ratio. If all drainage basin outlets have the same elevation, then chi values should be roughly the same on both sides of the divides in a steady-state situation. In this case, no drainage basin reorganization should be expected. Yet, if chi values differ, then one should expect that divides migrate from the basins with lower chi values (i.e. the aggressors) towards those with higher chi values (i.e. the victims) (see Figure below).

![](files/willett2014_fig1.png)

Now how is it possible to calculate chimaps using TopoToolbox? It is actually pretty easy, but there is no ready-made function. And there is a reason for this. The description above shows that plotting chimaps involves several modular steps, each of which should be carefully done. These steps are:

1. Load DEM and derive flow directions (FLOWobj).
2. Choose a suitable threshold for stream initiation and derive a stream network (STREAMobj).
3. Make sure that all outlets have the same elevation.
4. Make sure that all drainagebasins are complete, i.e. that the DEM covers their entire extent. Remove incomplete drainage basins.
5. Choose a suitable value for the mn-ratio.
6. Calculate the transformed variable chi (Hint: Don’t use chiplot, but check the low-level function STREAMobj/chitransform!).
7. Plot the results.


I will use a DEM of the Mendocino region (northwestern California) provided by Amani Al Abri from the University of Boston. Her inquiry has actually prompted me to write about chimaps. The Mendocino region hosts the Mendocino Triple Junction where the Gorda plate, the North American plate, and the Pacific plate meet. The geometry of the plates and their movements have likely favored the development of a slab window and upwelling of astenospheric material, leading to differential patterns of uplift, seismic activity and extrusion of volcanic rock.

## Load DEM and derive flow directions

In [ ]:
DEM = topo.read_tif('files/mendocino2.tif')

The first step is to load the DEM. While this is a trivial task, let me briefly mention a few pitfalls one may encounter. While the constructor function GRIDobj tries to identify missing values (actually, it does so quite reliably with the hack of Simon (thanks!)), I usually test this using the function

In [ ]:
# TODO: Show min/max elevation in info
DEM.info()

which provides several information including the minimum and maximum value. If these values are far beyond expected values, they should be replaced by NaNs.

Moreover, the info function shows the coordinate system of the DEM. TopoToolbox requires projected coordinate systems (preferably WGS84 UTM) and equal horizontal and vertical units (e.g. meters). If your DEM comes in geographic coordinates, reproject the DEM with the function reproject2utm. Note also, that you should set missing values before reprojecting the DEM since the bilinear interpolation would generate some blurred regions between non-corrected missing-value regions and regions with valid values.

Missing values usually occur at the boundaries of the DEM or offshore. These are truly missing values that you may want discard. However, data gaps may also exist as individual blobs within the valid DEM domain either as single pixels or large areas of missing values. These areas should be filled because they might otherwise generate discontinuous flow paths. Unless they are too large, I usually use the function inpaintnans to fill these gaps.

In [ ]:
# TODO: Implement inpaintnans
DEM.inpaintnans()

Having prepared the DEM, it is now time to derive flow directions. TopoToolbox stores flow directions in an object called FLOWobj.

In [ ]:
FD = topo.FlowObject(DEM)

FLOWobjs contain all necessary information to calculate hydrological terrain attributes (see the [function help](https://topotoolbox.github.io/pytopotoolbox/_autosummary/topotoolbox.FlowObject.html#topotoolbox.FlowObject) for details and some more detailed description on carving and filling [here](https://topotoolbox.wordpress.com/2014/04/26/the-difference-between-carving-and-filling-1/), [here](https://topotoolbox.wordpress.com/2014/05/04/the-difference-between-carving-and-filling-2/) and [here](https://topotoolbox.wordpress.com/2014/05/13/the-difference-between-carving-and-filling-3/)).

## Choose a suitable threshold for stream initiation and derive a stream network

Now that you have a DEM and flow directions, you can derive a stream network. TopoToolbox stores stream networks in a STREAMobj. In essence, a STREAMobj is a reduced version of the FLOWobj being limited to the channelized domain. Unless you have the locations of mapped channelheads, the standard approach to determine the channelized domain is to assume a minimum upslope area (MUA). You should neither choose a too large nor too small MUA. A large MUA may hide chimap details close to catchment divides; a small MUA may extend from the channelized domain into the debris flow or hillslope domain. Here we simply take an upslope area of 0.05 sqkm or 500 pixels:

In [ ]:
S = topo.StreamObject(FD,units='km2',threshold=0.05)
# TODO: This is very hard to see. Hillshade?
S.show(overlay=DEM)

## Make sure all outlets have the same elevation

Chimaps require the stream network to have outlets with the same elevation. As DEMs often fail to cover the entire altitude range of all individual streams it may be necessary to trim the stream network in a way so that all relevant streams have the same outlet elevation. The function GRIDobj/griddedcontour and STREAMobj/modify come in handy here (see also this [post](https://topotoolbox.wordpress.com/2015/06/22/demo-on-stream-network-modification/)). The modify function is extremely versatile. Here we use its option ‘upstreamto’ that requires a GRIDobj of zeros and ones. The modify function will then modify the stream network so that it contains streams only upstream of regions with ones. We use the output of griddedcontour and modify it slightly using the function bwmorph to produce gridded contours that are [four-connected](https://en.wikipedia.org/wiki/Pixel_connectivity#4-connected). A four-connected contour line ensures that all streams crossing the contours are reliably detected.

In [ ]:
# TODO: Implement these processing functions
C = DEM.griddedcontour(levels=[50],connectivity='4')
C.Z = bwmorph(C.Z,'diag')
S = S.modify(S,upstreamto=C)
C.show()
S.show(overlay=DEM)

## Make sure that all drainage basins are complete

Some of the drainage basins may be located along the DEM borders and have part of their area outside the DEM domain. To avoid edge effects on the resulting chimap, these incomplete drainage basins and their associated streams should be removed from further analysis. Testing whether a drainage basin is complete, however, is not trivial and the approach used here may not be useful in all situations.

Here I choose to identify incomplete basins by testing if their boundaries touch NaN areas or DEM edges. This requires some hard-coding since there is no ready-made function in TopoToolbox so far.

In [ ]:
# Get drainage basins
# TODO: Implement drainage basins
D = FD.drainagebasins(S);
 
# Get NaN mask and dilate it by one pixel.
I = np.isnan(DEM.z);
I = dilate(I,ones(3));
 
# Add border pixels to the mask
I.z[[1,-1],:] = true;
I.z[:,[1,-1]] = true;
 
# Get outlines for each basin
OUTLINES = np.zeros(DEM.shape,dtype=bool)
for r in 1..max(D):
    OUTLINES = OUTLINES | bwperim(D.Z == r);
 
# Calculate the fraction that each outline shares with the NaN mask
frac = accumarray(D.Z(OUTLINES),I.Z(OUTLINES),[],np.mean);
 
# Grid the fractions
FRAC = GRIDobj(DEM);
FRAC.z[:,:] = nan;
FRAC.z[D.z>0] = frac[D.z[D.z>0]];
FRAC.show()

The resulting GRIDobj FRAC is shown in Fig. 3. Several basins share a large proportion of their outlines with the DEM boundary and we don’t want these to be part of the analysis.

Based on the GRIDobj FRAC we now modify the STREAMobj so that it contains only streams where drainage basins share less than 1% with the NaN mask. Again, we use the function modify:

In [ ]:
S = S.modify(upstreamto=FRAC<=0.01);
S.show(overlay=DEM);

The trimmed stream network is shown above. Unfortunately, a large portion of the network was removed.

## The mathematical basis

Chi-analysis is predicated on the well-known stream power law (SPL). I will not detail the SPL but refer you to the paper of Dimitri Lague ([Lague 2014](http://onlinelibrary.wiley.com/doi/10.1002/esp.3462/abstract)). In its simplest form, the SPL shows that the energy expenditure required to incise into the stream bed is a function of stream gradient (S) and upslope area (A) and some parameters k, m and n. Combining the SPL with a simple vertical uplift rate U allows us to derive a very simple landscape evolution model for the fluvial domain:

$$
\frac{\text{d}z}{\text{d}t} = U - kA^mS^n
$$

Any change in elevation z with time t is caused by an imbalance of uplift and incision. If both processes are perfectly balanced, there is no change in elevation. We are in a state of a dynamic equilibrium or steady state:

$$
U - kA^mS^n = 0
$$

We can rearrange this equation and solve for S=dz/dx where x is the stream distance measured from the outlet. At this time I denote that upslope area is a function of x. U and k are assumed to be spatially uniform.

$$
\frac{\text{d}z}{\text{d}x} = \left(\frac{U}{kA(x)^m}\right)^{1/n}
$$

The trick is then to take the integral of ($dz/dx$) with respect to $x$ from a base level $z(x_0)$ (the elevation at the outlet). For convenience of units, we also introduce a reference area $A_0$.

$$
\int \frac{\text{d}z}{\text{d}x}\ \text{d}x = z(x_0) + \left(\frac{U}{kA_0^m}\right)^{1/n} \int \frac{A_0}{A(x)^{m/n}}\ \text{d}x
$$

Now that looks complicated. Yet, by replacing the left-hand integral with z and the right-hand integral with

$$
\chi = \int \frac{A_0}{A(x)^{m/n}}\ \text{d}x
$$

we can simply rewrite this equation as a function of a straight line:

$$
z = z(x_0) + \left(\frac{U}{kA_0^m}\right)^{1/n} \chi
$$

A straight line? Yes… but only if we choose the right m/n ratio.

This was really a short derivation of chi. I can recommend the [paper of Perron and Royden (2013)](http://onlinelibrary.wiley.com/doi/10.1002/esp.3302/abstract) for a more comprehensive account.

### References

Lague, D.: The stream power river incision model: evidence, theory and beyond, Earth Surf. Process. Landforms, 39(1), 38–61, doi:10.1002/esp.3462, 2014.

Perron, J. T. and Royden, L.: An integral approach to bedrock river profile analysis, Earth Surf. Process. Landforms, 38(6), 570–576, doi:10.1002/esp.3302, 2013.


## Finding the right m/n ratio

By using the integral approach to the stream power model of incision, we derived an equation that allows us to model the longitudinal river profile as a function of chi. I stated that this model is a straight line if we choose the right m/n ratio, that is the concavity index. I’ll show how we can obtain this m/n ratio by means of nonlinear optimization using the function chiplot.

Ideally, we find the m/n ratio using a perfectly graded stream profile that is in steady state. I scanned through a number of streams using the GUI flowpathapp and found a nice one that might correspond to Cooskie Creek that Perron and Royden (2013) used in their analysis.

In [ ]:
# This won't work, but probably shouldn't...
flowpathapp(FD,DEM,S)

The menu Export > Export to workspace allows saving the extracted STREAMobj St to the workspace. Then, I use the function chiplot to see how different values of the m/n ratio affect the transformation of the river profile. Setting the ‘mnplot’ option makes chiplot return a figure with chi-transformed profiles for m/n ratios ranging from 0.1 to 0.9 in steps of 0.1 width.

In [ ]:
S.chiplot(DEM,A,mnplot=true,plot=false)

Clearly, the chi-transformed profile varies from concave upward for low m/n values to convex for high m/n values. A value of 0.4-0.5 seems to be most appropriate but choosing a value would be somewhat hand-wavy. Thus, let’s call the function again, but this time without any additional arguments.

In [ ]:
C = S.chiplot(DEM,A);

Calling the function this way will prompt it to find an optimal value of m/n. In addition, it will return a figure with the linearized profile. By default, the function uses the optimization function fminsearch that will vary m/n until it finds a value that maximizes the linear correlation coefficient between the chi-transformed profile and a straight line. So what is that value? Let’s look at the output variable C:

In [ ]:
C

C is a structure array with a large number of fields. C.mn is the value we are interested in. It is 0.4776 and corresponds nicely to previously reported m/n ratios although it differs from the value 0.36 found by Perron and Royden (2013). Of course, this value might differ from river to river and you should repeat the analysis for other reaches to obtain an idea about the variability of m/n.

Ok, we have the m/n ratio now. Next we will eventually produce the chimap that we initially set out for.

P.S.: We could have also used slope-area plots to derive the m/n ratio (see the function slopearea). However, along-river gradients are usually much more noisy than the profiles and power-law fitting a delicate matter. I’d definitely prefer chi-analysis over slope-area analysis.

## Plotting the chi-map

Having prepared a stream network and equipped with a reasonable value of the m/n ratio, we are now ready to plot a chimap that visualizes the planform patterns of chi. The main interest in these maps lies in chi values near catchment divides as large differences between adjacent catchment would indicate a transient behavior of drainage basin reorganization.

Some of you might have already experimented with TopoToolbox to create chimaps. Perhaps you became exasperated with the function chiplot that is restricted to calculations with only one drainage basin and has a bewildering structure array as output. The reason for the confusing output of chiplot is that it is fairly old. At this time, I hadn’t implemented node-attribute lists that are now more common with STREAMobj methods.

Realizing this shortcoming of chiplot, I wrote the function chitransform. chitransform is what I’d refer to as a low-level function that solves the chi-equation using upstream cumulative trapezoidal integration (see the function cumtrapz). chitransform requires a STREAMobj and a flow accumulation grid as input and optionally a mn-ratio (default is 0.45) and a reference area (default is 1 sqkm). It returns a node-attribute list, i.e., a vector with chi values for each node in the STREAMobj. Node-attribute lists are intrinsically tied to the STREAMobj from which they were derived. Yet, they can be used together with several other TopoToolbox functions to produce output.

Ok, let’s derive chi values for our stream network:

In [ ]:
A = FD.flow_accumulation()

In [ ]:
c = S.chitransform(A,mn=0.4776);

In the next step, we will plot a color stream network on a grayscale hillshade:

In [ ]:
S.show(overlay=DEM,color=c)

Interestingly, there seem to be some locations with quite some differences in chi values on either side of the divide. “Victims” seem to be rather elongated catchments draining northwest. Let’s zoom into one of these locations.

In [ ]:
# TODO: Zoom in to this location

Are these significant differences? Well, it seems by just looking at the range of values. However, to my knowledge no approach exists that provides a more objective way of evaluating the significance of contrasting chi values and their implications about rates of divide migration. Still, we now have a nice map that can aid our geomorphic assessment together with the tectonic and geodynamic interpretation of the Mendocino Triple Junction.

Unfortunately, I must leave the discussion to you since I am quite unfamiliar with the region. If anyone wants to share his or her interpretation, I’d be more than happy to provide space here. So far, I hope that these few posts on chimaps were useful to you and sufficiently informative to enable you to compute chimaps by yourself. In my next post, I will give a short summary and show with another example that eventually chimaps can be derived really in a few lines of code.